In [1]:
import csv
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm

import matplotlib.patches as mpatches

from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression

from IPython.core.debugger import Pdb #Pdb().set_trace()
from collections import Counter


pd.set_option('display.max_columns', None)

coeffs = ['IP', 'BT', 'NEL', 'PLTH', 'RGEO', 'KAREA', 'EPS', 'MEFF']
path = "../data/"

In [2]:
DB5 = pd.read_excel(path+"HDB5V2.3.xlsx")

In [3]:
STDB5 = DB5[DB5["SELDB5"].isin([1])]
# ELMy H
STDB5 = STDB5[STDB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]
STDB5

,TOK,TOK_ID,DIVNAME,LCUPDATE,DATE,SHOT,TIME,TIME_ID,T1,T2,AUXHEAT,PHASE,HYBRID,ITB,ITBTYPE,ELMTYPE,ELMFREQ,ELMMAX,ELMDUR,ELMINT,OLTIME,LHTIME,TPI,ISEQ,MEFF,PGASA,PGASZ,BGASA,BGASZ,BGASA2,BGASZ2,PELLET,FUELRATE,XGASZ,XGASA,CONFIG,RGEO,RMAG,AMIN,KAPPA,KAPPAA,KAREA,DELTA,DELTAU,DELTAL,INDENT,AREA,VOL,SURFFORM,SEPLIM,XPLIM,WALMAT,DIVMAT,LIMMAT,EVAP,DALFMP,DALFDV,IGRADB,BT,IEML,PREMAG,IP,VSURF,Q95,SH95,BEILI2,BEIMHD,BEPMHD,BETMHD,BEPDIA,BMHDMDIA,TAUCR,FBS,RHOQ2,RHOINV,NEL,NELFORM,DNELDT,NEV,NE0,NE0TSC,ZEFF,ZEFFNEO,PRAD,POHM,ENBI,PINJ,BSOURCE,PINJ2,BSOURCE2,COCTR,PNBI,PFLOSS,ECHFREQ,ECHMODE,ECHLOC,PECRHC,PECRH,ICFREQ,ICSCHEME,ICANTEN,PICRHC,PICRH,PALPHA,DWDIA,DWDIAPAR,DWMHD,DWHC,TEV,TE0,TE0TSC,TIV,TI0,TICX0,WDIA,WMHD,WKIN,WEKIN,WIKIN,WROT,WFPER,WFPAR,WFFORM,WFANI,WFICRH,WFICRHP,WFICFORM,ICFORM,WFANIIC,TAUDIA,TAUMHD,TAUTH1,TAUTH2,WTOT,WTH,PL,PLTH,TAUTOT],TAUTH,TAUC92,TAUC93,H89,HITER96L,H93,HITER92Y,HEPS97,HIPB98Y,HIPB98Y1,HIPB98Y2,HIPB98Y3,HIPB98Y4,OMGAIMP0,OMGAIMPH,OMGAM0,OMGAMH,SPIN,TORQ,TORQBM,TORQIN,VTOR0,VTORV,VTORIMP,STANDARD,SELDB1,SELDB2,SELDB2X,IAEA92,DB2P5,DB2P8,DB3IS,DB3V5,IAE2000N,IAE2000X,HMWS2003,IAE2004S,IAE2004I,DB3DONLY,HMWS2005,OJK2006,SELDB3,SELDB3X,SELDB4,AAREA,STDDB4V5,NESEP,NESOL,PMAIN,PDIV,GP_MAIN,GP_DIV,SELDB5,STD3
5,ASDEX,1,DV-IPRE,19940125,19820622,5980,1.205,1205,1.203,1.207,NB,HGELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.110,1.166,6.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.6940,1.8030,0.4101,0.9752,0.975092,0.9748,0.0000,0.0000,0.00000,0.0,0.5152,5.4820,27.088080,0.15640,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,295900.0,0.23200,4.563,NaN,2.649,2.0190,2.019,0.009265,1.6670,0.4481,5.423056,NaN,NaN,0.277121,37890000000000000000,0.0,-19590000000000000000,30170000000000000000,51000000000000000000,,NaN,NaN,0.0,68660.0,42000.00,2620000.0,404020.0,0.0,0.0,1.0000,2599000.0,585700.0,0.000000e+00,NONE,NONE,0.0,0.0,0.0,NONE,NONE,0.0,0.0,0.0,212000.0,620500.0,212000.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,116400.0,141100.0,NaN,NaN,NaN,NaN,NaN,NaN,24260.0,0.5780,0.0,0.0,0.0,0.0,0.0,0.04746,0.05753,NaN,0.05100,119600.0,95370.0,2456000.0,1870000.0,0.04871,0.05100,0.6667,0.6667,1.612,1.413,0.9272,1.103,1.126,1.150,1.119,1.132,1.130,1.0110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,111.0,1.111111e+09,1.111111e+09,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.111111e+09,111100.0,0.0,NaN,NaN,,,NaN,NaN,NaN,NaN,1,1.0
6,ASDEX,1,DV-IPRE,19940125,19820622,5980,1.224,1224,1.222,1.226,NB,HGELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.110,1.166,7.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.6840,1.7970,0.4113,0.9724,0.972233,0.9724,0.0000,0.0000,0.00000,0.0,0.5167,5.4680,26.969199,0.16430,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,295200.0,0.07778,4.671,NaN,2.720,2.0900,2.090,0.009540,1.7510,0.4348,16.042211,NaN,NaN,0.273998,37340000000000000000,0.0,-44680000000000000000,29400000000000000000,52000000000000000000,,NaN,NaN,0.0,22960.0,42000.00,2620000.0,404020.0,0.0,0.0,1.0000,2598000.0,596700.0,0.000000e+00,NONE,NONE,0.0,0.0,0.0,NONE,NONE,0.0,0.0,0.0,0.0,-485.4,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,121100.0,144500.0,NaN,NaN,NaN,NaN,NaN,NaN,25220.0,0.5782,0.0,0.0,0.0,0.0,0.0,0.04628,0.05522,NaN,0.04902,124400.0,99230.0,2621000.0,2024000.0,0.04748,0.04902,0.6667,0.6667,1.641,1.470,0.9572,1.139,1.165,1.185,1.152,1.169,1.167,1.0440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,111.0,1.111111e+09,1.111111e+09,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.111111e+09,111100.0,0.0,NaN,NaN,,,NaN,NaN,NaN,NaN,1,1.0
11,ASDEX,1,DV-IPRE,19940125,19820622,5982,1.188,1188,1.186,1.190,NB,HSELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.115,1.165,5.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.6930,1.7870,0.4043,0.9895,0.989248,0.9895,0.0000,0.0000,0.00000,0.0,0.5080,5.4050,26.880691,0.16230,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,297100.0,-0.05525,4.275,NaN,2.289,1.6600,1.660,0.007679,1.3130,0.4427,22.850781,NaN,NaN,0.282055,34100000000000000000,0.0,192600000000000000000,28300000000000000000,45000000000000000000,,NaN,

In [4]:
STDB2 = DB5[DB5.SELDB2X > 0]
STDB2 = STDB2[STDB2["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]
STDB2

,TOK,TOK_ID,DIVNAME,LCUPDATE,DATE,SHOT,TIME,TIME_ID,T1,T2,AUXHEAT,PHASE,HYBRID,ITB,ITBTYPE,ELMTYPE,ELMFREQ,ELMMAX,ELMDUR,ELMINT,OLTIME,LHTIME,TPI,ISEQ,MEFF,PGASA,PGASZ,BGASA,BGASZ,BGASA2,BGASZ2,PELLET,FUELRATE,XGASZ,XGASA,CONFIG,RGEO,RMAG,AMIN,KAPPA,KAPPAA,KAREA,DELTA,DELTAU,DELTAL,INDENT,AREA,VOL,SURFFORM,SEPLIM,XPLIM,WALMAT,DIVMAT,LIMMAT,EVAP,DALFMP,DALFDV,IGRADB,BT,IEML,PREMAG,IP,VSURF,Q95,SH95,BEILI2,BEIMHD,BEPMHD,BETMHD,BEPDIA,BMHDMDIA,TAUCR,FBS,RHOQ2,RHOINV,NEL,NELFORM,DNELDT,NEV,NE0,NE0TSC,ZEFF,ZEFFNEO,PRAD,POHM,ENBI,PINJ,BSOURCE,PINJ2,BSOURCE2,COCTR,PNBI,PFLOSS,ECHFREQ,ECHMODE,ECHLOC,PECRHC,PECRH,ICFREQ,ICSCHEME,ICANTEN,PICRHC,PICRH,PALPHA,DWDIA,DWDIAPAR,DWMHD,DWHC,TEV,TE0,TE0TSC,TIV,TI0,TICX0,WDIA,WMHD,WKIN,WEKIN,WIKIN,WROT,WFPER,WFPAR,WFFORM,WFANI,WFICRH,WFICRHP,WFICFORM,ICFORM,WFANIIC,TAUDIA,TAUMHD,TAUTH1,TAUTH2,WTOT,WTH,PL,PLTH,TAUTOT],TAUTH,TAUC92,TAUC93,H89,HITER96L,H93,HITER92Y,HEPS97,HIPB98Y,HIPB98Y1,HIPB98Y2,HIPB98Y3,HIPB98Y4,OMGAIMP0,OMGAIMPH,OMGAM0,OMGAMH,SPIN,TORQ,TORQBM,TORQIN,VTOR0,VTORV,VTORIMP,STANDARD,SELDB1,SELDB2,SELDB2X,IAEA92,DB2P5,DB2P8,DB3IS,DB3V5,IAE2000N,IAE2000X,HMWS2003,IAE2004S,IAE2004I,DB3DONLY,HMWS2005,OJK2006,SELDB3,SELDB3X,SELDB4,AAREA,STDDB4V5,NESEP,NESOL,PMAIN,PDIV,GP_MAIN,GP_DIV,SELDB5,STD3
5,ASDEX,1,DV-IPRE,19940125,19820622,5980,1.205,1205,1.203,1.207,NB,HGELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.110,1.166,6.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.694,1.803,0.4101,0.9752,0.975092,0.9748,0.0,0.0,0.0,0.0,0.5152,5.482,27.088080,0.1564,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,295900.0,0.23200,4.563,NaN,2.649,2.019,2.019,0.009265,1.667,0.4481,5.423056,NaN,NaN,0.277121,37890000000000000000,0.0,-19590000000000000000,30170000000000000000,51000000000000000000,,NaN,NaN,0.0,68660.0,42000.0,2620000.0,404020.0,0.0,0.0,1.0,2599000.0,585700.0,0.0,NONE,NONE,0.0,0.0,0.0,NONE,NONE,0.0,0.0,0.0,212000.00,620500.0,212000.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,116400.0,141100.0,NaN,NaN,NaN,NaN,NaN,NaN,24260.0,0.5780,0.0,0.0,0.0,0.0,0.0,0.04746,0.05753,NaN,0.05100,119600.0,95370.0,2456000.0,1870000.0,0.04871,0.05100,0.6667,0.6667,1.612,1.413,0.9272,1.1030,1.1260,1.1500,1.1190,1.1320,1.1300,1.0110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,111.0,1.111111e+09,1.111111e+09,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.111111e+09,111100.0,0.0,NaN,NaN,,,NaN,NaN,NaN,NaN,1,1.0
6,ASDEX,1,DV-IPRE,19940125,19820622,5980,1.224,1224,1.222,1.226,NB,HGELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.110,1.166,7.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.684,1.797,0.4113,0.9724,0.972233,0.9724,0.0,0.0,0.0,0.0,0.5167,5.468,26.969199,0.1643,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,295200.0,0.07778,4.671,NaN,2.720,2.090,2.090,0.009540,1.751,0.4348,16.042211,NaN,NaN,0.273998,37340000000000000000,0.0,-44680000000000000000,29400000000000000000,52000000000000000000,,NaN,NaN,0.0,22960.0,42000.0,2620000.0,404020.0,0.0,0.0,1.0,2598000.0,596700.0,0.0,NONE,NONE,0.0,0.0,0.0,NONE,NONE,0.0,0.0,0.0,0.00,-485.4,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,121100.0,144500.0,NaN,NaN,NaN,NaN,NaN,NaN,25220.0,0.5782,0.0,0.0,0.0,0.0,0.0,0.04628,0.05522,NaN,0.04902,124400.0,99230.0,2621000.0,2024000.0,0.04748,0.04902,0.6667,0.6667,1.641,1.470,0.9572,1.1390,1.1650,1.1850,1.1520,1.1690,1.1670,1.0440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,111.0,1.111111e+09,1.111111e+09,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.111111e+09,111100.0,0.0,NaN,NaN,,,NaN,NaN,NaN,NaN,1,1.0
11,ASDEX,1,DV-IPRE,19940125,19820622,5982,1.188,1188,1.186,1.190,NB,HSELM,UNKNOWN,UNKNOWN,NaN,UNKNOWN,NaN,NaN,NaN,NaN,1.115,1.165,5.0,NONE,1.5,2.0,1,1,1,0,0,NONE,,0,0.0,DN,1.693,1.787,0.4043,0.9895,0.989248,0.9895,0.0,0.0,0.0,0.0,0.5080,5.405,26.880691,0.1623,NaN,SS,TI2,NONE,NONE,NaN,NaN,NaN,2.205,0.0,NaN,297100.0,-0.05525,4.275,NaN,2.289,1.660,1.660,0.007679,1.313,0.4427,22.850781,NaN,NaN,0.282055,34100000000000000000,0.0,192600000000000000000,28300000000000000000,45000000000000000000,,NaN,NaN,0.0,0.0,42000.0,2620000.0,404020.0,0.0,0.0,1.0,2590000.0

In [5]:
STDB2["SELDB2X"].unique()

array([1.11111111e+09, 1.11101111e+09, 1.11001111e+09, 1.11011111e+09,
       1.11111111e+08, 1.11011111e+08, 1.01101111e+09, 1.01111111e+09,
       1.01011111e+09, 1.01001111e+09, 1.11111011e+09, 1.11011011e+09,
       1.11111110e+09, 1.10111111e+08, 1.11101011e+09, 1.11100011e+09,
       1.11100111e+09, 1.11000011e+09, 1.11001011e+09, 1.11000111e+09,
       1.11101110e+09, 1.10111011e+09, 1.10111010e+09, 1.10101011e+09,
       1.10101110e+09, 1.10110111e+09, 1.10111111e+09, 1.10100110e+09,
       1.10101111e+09, 1.10100111e+09, 1.10100011e+09, 1.11100110e+09,
       1.10111110e+09, 1.10001110e+09, 1.11110011e+09, 1.11110111e+09,
       1.11101111e+09, 1.11111111e+09])

In [7]:
DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False)

# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

In [9]:
Y_ = DB5[["TAUTH"]].apply(np.log).to_numpy()
# Adding a column for the intercept
_df = DB5[coeffs].apply(np.abs).apply(np.log)
_df.insert(
    loc = 0, 
    column = "intercept", 
    value = np.ones(len(_df))
)
X_ = _df.to_numpy()
n_, p_ = X_.shape
model = sm.OLS(Y_,X_)
regression = model.fit()
regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.954
Model:                            OLS   Adj. R-squared:                  0.954
Method:                 Least Squares   F-statistic:                 1.955e+04
Date:                Fri, 31 Mar 2023   Prob (F-statistic):               0.00
Time:                        20:40:05   Log-Likelihood:                 1256.6
No. Observations:                7556   AIC:                            -2495.
Df Residuals:                    7547   BIC:                            -2433.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.5434      0.021   -119.448      0.000      -2.585      -2.502
x1             1.0704      0.015     72.213      0.000       1.041       1.099
x2             0.1619      0.012     13.185      0.000       0.138       0.186
x3             0.1905      0.009     21.778      0.000       0.173       0.208
x4            -0.6992      0.006   -120.740      0.000      -0.711      -0.688
x5             1.5863      0.023     69.656      0.000       1.542       1.631
x6             0.4371      0.025     17.793      0.000       0.389       0.485
x7             0.2265      0.030      7.577      0.000       0.168       0.285
x8             0.2257      0.015     14.994      0.000       0.196       0.255
==============================================================================
Omnibus:                      138.596   Durbin-Watson:                   0.564
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              200.933
Skew:                          -0.210   Prob(JB):                     2.33e-44
Kurtosis:                       3.679   Cond. No.                         58.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
np.exp(-2.5434)

0.07859870936298877

In [1]:
"BRIEF LECTURES ON QUANTUM COMPUTATION".title()

'Brief Lectures On Quantum Computation'